# Get Women's Matchups

Get head-to-head records and records against common opponents

### Head-to-Head

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 100)

include_2024 = True

df = pd.concat(
    [
        pd.read_csv(fr'..\data\unprocessed\womens_sports_reference\womens_full_year_sports_reference_{season}.csv')
        .assign(Season=season)
        for season in range(2012, 2024 + include_2024)
        if season != 2020  # tournament was cancelled
    ],
    ignore_index=True
)

df.insert(0, 'Season', df.pop('Season'))

df = df.loc[df['NCAA Tournament'] == 0, :].reset_index(drop=True)  # only include games before NCAA Tournament

df

,Season,Date,NCAA Tournament,Team,Location,Opponent,Result,Team Score,Opponent Score,Team FG,Team FGA,Team FG%,Team 3P,Team 3PA,Team 3P%,Team FT,Team FTA,Team FT%,Team ORB,Team TRB,Team AST,Team STL,Team BLK,Team TOV,Team PF,Opponent FG,Opponent FGA,Opponent FG%,Opponent 3P,Opponent 3PA,Opponent 3P%,Opponent FT,Opponent FTA,Opponent FT%,Opponent ORB,Opponent TRB,Opponent AST,Opponent STL,Opponent BLK,Opponent TOV,Opponent PF,Overtimes Amount,Overtime,Score Differential,Adjusted Score Differential,Possessions,Team PPP,Opponent PPP,Tempo
0,2012,2011-11-11,0,Air Force,-1,Lipscomb,1,68.0,59.0,23.0,63.0,0.365,1.0,4.0,0.250,21.0,30.0,0.700,15.0,41.0,11.0,11.0,2.0,16.0,23.0,18.0,54.0,0.333,4.0,12.0,0.333,19.0,31.0,0.613,13.0,36.0,10.0,8.0,3.0,23.0,24.0,0,0,9.0,1.361013,77.42,0.878326,0.762077,77.42
1,2012,2011-11-13,0,Air Force,-1,Tennessee-Martin,-1,60.0,84.0,27.0,64.0,0.422,4.0,8.0,0.500,2.0,4.0,0.500,12.0,35.0,17.0,10.0,1.0,27.0,16.0,32.0,69.0,0.464,10.0,30.0,0.333,10.0,11.0,0.909,9.0,31.0,15.0,12.0,4.0,18.0,7.0,0,0,-24.0,1.500000,81.80,0.733496,1.026895,81.80
2,2012,2011-11-17,0,Air Force,1,Denver,-1,52.0,54.0,20.0,60.0,0.333,1.0,8.0,0.125,11.0,20.0,0.550,14.0,48.0,15.0,7.0,3.0,21.0,26.0,20.0,57.0,0.351,1.0,10.0,0.100,13.0,30.0,0.433,10.0,42.0,16.0,13.0,4.0,20.0,19.0,0,0,-2.0,1.102120,78.00,0.666667,0.692308,78.00
3,2012,2011-11-19,0,Air Force,-1,Texas Southern,1,64.0,54.0,20.0,51.0,0.392,0.0,2.0,0.000,24.0,33.0,0.727,8.0,35.0,13.0,14.0,1.0,21.0,20.0,17.0,57.0,0.298,5.0,18.0,0.278,15.0,24.0,0.625,15.0,41.0,13.0,10.0,6.0,30.0,27.0,0,0,10.0,1.381279,80.54,0.794636,0.670474,80.54
4,2012,2011-11-21,0,Air Force,-1,Texas State,1,77.0,69.0,25.0,61.0,0.410,1.0,6.0,0.167,26.0,31.0,0.839,16.0,45.0,10.0,9.0,3.0,19.0,21.0,27.0,67.0,0.403,4.0,21.0,0.190,11.0,20.0,0.550,13.0,30.0,10.0,8.0,2.0,14.0,20.0,0,0,8.0,1.338710,77.22,0.997151,0.893551,77.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122551,2024,2024-02-24,0,Youngstown State,-1,Northern Kentucky,1,68.0,64.0,24.0,57.0,0.421,12.0,23.0,0.522,8.0,11.0,0.727,8.0,36.0,12.0,4.0,6.0,13.0,14.0,22.0,61.0,0.361,10.0,23.0,0.435,10.0,13.0,0.769,8.0,27.0,13.0,10.0,4.0,8.0,12.0,0,0,4.0,1.214668,66.78,1.018269,0.958371,66.78
122552,2024,2024-02-27,0,Youngstown State,1,Cleveland State,-1,53.0,56.0,21.0,60.0,0.350,7.0,26.0,0.269,4.0,4.0,1.000,5.0,34.0,15.0,10.0,4.0,19.0,20.0,17.0,58.0,0.293,4.0,13.0,0.308,18.0,22.0,0.818,12.0,43.0,5.0,7.0,1.0,17.0,11.0,0,0,-3.0,1.166625,74.22,0.714093,0.754514,74.22
122553,2024,2024-03-02,0,Youngstown State,-1,Purdue Fort Wayne,-1,64.0,73.0,23.0,56.0,0.411,7.0,21.0,0.333,11.0,12.0,0.917,5.0,29.0,16.0,10.0,6.0,16.0,15.0,27.0,62.0,0.435,13.0,34.0,0.382,6.0,9.0,0.667,9.0,35.0,12.0,7.0,2.0,14.0,15.0,0,0,-9.0,1.361013,71.62,0.893605,1.019268,71.62
122554,2024,2024-03-05,0,Youngstown State,1,IUPUI,1,73.0,50.0,29.0,63.0,0.460,8.0,16.0,0.500,7.0,11.0,0.636,7.0,30.0,20.0,9.0,4.0,12.0,22.0,16.0,48.0,0.333,1.0,12.0,0.083,17.0,25.0,0.680,6.0,32.0,12.0,7.0,8.0,18.0,19.0,0,0,23.0,1.500000,71.92,1.015017,0.695217,71.92


In [2]:
df_h2h = (
    df
    .groupby(['Season', 'Team', 'Opponent'])
    ['Result']
    .agg(['sum', 'count'])
    .reset_index()
    .rename(columns={'Team': 'Team A', 'Opponent': 'Team B'})
)

df_h2h['Head to Head'] = df_h2h['sum'] / (df_h2h['count'] + 1/4)  # add 1/4 to denominator so that a team that beats opponent multiple times is better accounted for

df_h2h

,Season,Team A,Team B,sum,count,Head to Head
0,2012,Air Force,Alabama,-1,1,-0.800000
1,2012,Air Force,Boise State,-2,2,-0.888889
2,2012,Air Force,Colorado State,-2,2,-0.888889
3,2012,Air Force,Denver,-1,1,-0.800000
4,2012,Air Force,Jackson State,1,1,0.800000
...,...,...,...,...,...,...
86791,2024,Youngstown State,St. Bonaventure,-1,1,-0.800000
86792,2024,Youngstown State,West Virginia,-1,1,-0.800000
86793,2024,Youngstown State,Western Michigan,-1,1,-0.800000
86794,2024,Youngstown State,Wright State,0,2,0.000000


### Common Opponents

In [3]:
df_h2h['Win Pct'] = ((df_h2h['sum'] / df_h2h['count']) + 1)/2

df_h2h

,Season,Team A,Team B,sum,count,Head to Head,Win Pct
0,2012,Air Force,Alabama,-1,1,-0.800000,0.0
1,2012,Air Force,Boise State,-2,2,-0.888889,0.0
2,2012,Air Force,Colorado State,-2,2,-0.888889,0.0
3,2012,Air Force,Denver,-1,1,-0.800000,0.0
4,2012,Air Force,Jackson State,1,1,0.800000,1.0
...,...,...,...,...,...,...,...
86791,2024,Youngstown State,St. Bonaventure,-1,1,-0.800000,0.0
86792,2024,Youngstown State,West Virginia,-1,1,-0.800000,0.0
86793,2024,Youngstown State,Western Michigan,-1,1,-0.800000,0.0
86794,2024,Youngstown State,Wright State,0,2,0.000000,0.5


In [4]:
def get_opponents_results(season, team):
    data = df_h2h.loc[
        (df_h2h['Season'] == season) & 
        (df_h2h['Team A'] == team), 
        :
    ]
    return dict(zip(
        data['Team B'],
        data['Win Pct']
    ))

In [5]:
from tqdm.autonotebook import tqdm

season_team_opponents = {
    season: {
        team: get_opponents_results(season, team) 
        for team in df_h2h.loc[df_h2h['Season'] == season, 'Team A'].unique()
    } 
    for season in tqdm(df_h2h['Season'].unique())
}

len(season_team_opponents)

C:\Users\mhugh\AppData\Local\Temp\ipykernel_4720\755660832.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/12 [00:00<?, ?it/s]

12

In [6]:
import numpy as np

def get_win_pct_difference(season, team_a, team_b):
    team_a_dict = season_team_opponents[season][team_a]
    team_b_dict = season_team_opponents[season][team_b]

    common_keys = team_a_dict.keys() & team_b_dict.keys()

    if len(common_keys) == 0:
        return np.nan
    
    commons_opps_results = tuple(team_a_dict[key] - team_b_dict[key] for key in common_keys)

    return np.sum(commons_opps_results)/(len(commons_opps_results) + 1/4)  # add 1/4 to denominator so that the amount of matchups is better accounted for

In [7]:
common_opps = [
    (season, team_a, team_b, get_win_pct_difference(season, team_a, team_b))
    for season in tqdm(df_h2h['Season'].unique())
    for team_a in df_h2h.loc[df_h2h['Season'] == season, 'Team A'].unique()
    for team_b in df_h2h.loc[df_h2h['Season'] == season, 'Team A'].unique()  # could use Team B but shouldn't matter
    if team_a != team_b
]

common_opps[:5]

  0%|          | 0/12 [00:00<?, ?it/s]

[(2012, 'Air Force', 'Akron', nan),
 (2012, 'Air Force', 'Alabama', nan),
 (2012, 'Air Force', 'Alabama A&M', 0.15384615384615385),
 (2012, 'Air Force', 'Alabama State', 0.0),
 (2012, 'Air Force', 'Albany (NY)', nan)]

In [8]:
df_common_opps = pd.DataFrame(
    common_opps,
    columns=['Season', 'Team A', 'Team B', 'Common Opps'],
)

df_common_opps

,Season,Team A,Team B,Common Opps
0,2012,Air Force,Akron,NaN
1,2012,Air Force,Alabama,NaN
2,2012,Air Force,Alabama A&M,0.153846
3,2012,Air Force,Alabama State,0.000000
4,2012,Air Force,Albany (NY),NaN
...,...,...,...,...
1467597,2024,Youngstown State,Wofford,NaN
1467598,2024,Youngstown State,Wright State,-0.097561
1467599,2024,Youngstown State,Wyoming,-0.400000
1467600,2024,Youngstown State,Xavier,NaN


In [9]:
df_h2h = pd.merge(
    df_h2h.drop(columns=['sum', 'count', 'Win Pct']),
    df_common_opps,
    how='outer',
    on=['Season', 'Team A', 'Team B'],
)

df_h2h

,Season,Team A,Team B,Head to Head,Common Opps
0,2012,Air Force,Alabama,-0.800000,NaN
1,2012,Air Force,Boise State,-0.888889,-0.160000
2,2012,Air Force,Colorado State,-0.888889,-0.288288
3,2012,Air Force,Denver,-0.800000,-0.941176
4,2012,Air Force,Jackson State,0.800000,0.266667
...,...,...,...,...,...
1467597,2024,Youngstown State,Winthrop,NaN,NaN
1467598,2024,Youngstown State,Wisconsin,NaN,-0.400000
1467599,2024,Youngstown State,Wofford,NaN,NaN
1467600,2024,Youngstown State,Wyoming,NaN,-0.400000


Remove all the rows without data to reduce the dataframe size

In [10]:
df_h2h = df_h2h.loc[(df_h2h['Head to Head'].notna()) | (df_h2h['Common Opps'].notna()), :].reset_index(drop=True)

df_h2h

,Season,Team A,Team B,Head to Head,Common Opps
0,2012,Air Force,Alabama,-0.800000,NaN
1,2012,Air Force,Boise State,-0.888889,-0.160000
2,2012,Air Force,Colorado State,-0.888889,-0.288288
3,2012,Air Force,Denver,-0.800000,-0.941176
4,2012,Air Force,Jackson State,0.800000,0.266667
...,...,...,...,...,...
736439,2024,Youngstown State,Weber State,NaN,0.000000
736440,2024,Youngstown State,Western Kentucky,NaN,0.400000
736441,2024,Youngstown State,Wichita State,NaN,0.000000
736442,2024,Youngstown State,Wisconsin,NaN,-0.400000


In [11]:
df_h2h.to_csv('../data/preprocessed/women_h2h/women_h2h.csv', index=False)

'Done'

'Done'